In [1]:
%matplotlib inline

In [2]:
import nltk

In [3]:
legal_pos = set(['n', 'v.t.', 'v.i.', 'adj', 'det'])

In [5]:
import re # added
pattern = re.compile(r"'font-size:11.0pt'>([a-z.]+)<")

In [6]:
document = open("dict.htm", encoding="windows-1252").read()

FileNotFoundError: [Errno 2] No such file or directory: 'dict.htm'

In [ ]:
used_pos = set(re.findall(pattern, document))

In [ ]:
illegal_pos = used_pos.difference(legal_pos)

In [ ]:
print(list(illegal_pos))
# ['v.i', 'intrans']

In [7]:
from bs4 import BeautifulSoup

def lexical_data(html_file, encoding="utf-8"):
    SEP = '_ENTRY'
    html = open(html_file, encoding=encoding).read()
    html = re.sub(r'<p', SEP + '<p', html)
    text = BeautifulSoup(html).get_text()
    text = ' '.join(text.split())
    for entry in text.split(SEP):
        if entry.count(' ') > 2:
            yield entry.split(' ', 3)

In [ ]:
import csv

In [ ]:
writer = csv.writer(open("dict1.csv", "w", encoding="utf-8"))

In [ ]:
writer.writerows(lexical_data("dict.htm", encoding="windows-1252"))

In [ ]:
import csv

In [ ]:
lexicon = csv.reader(open('dict.csv'))

In [ ]:
pairs = [(lexeme, defn) for (lexeme, _, _, defn) in lexicon]

In [ ]:
lexemes, defns = zip(*pairs)

In [ ]:
defn_words = set(w for defn in defns for w in defn.split())

In [ ]:
sorted(defn_words.difference(lexemes))
# ['...', 'a', 'and', 'body', 'by', 'cease', 'condition', 'down', 'each',
# 'foot', 'lifting', 'mind', 'of', 'progress', 'setting', 'to']

In [8]:
idx = nltk.Index((defn_word, lexeme)
 for (lexeme, defn) in pairs
 for defn_word in nltk.word_tokenize(defn)
 if len(defn_word) > 3)

NameError: name 'pairs' is not defined

In [ ]:
with open("dict.idx", "w") as idx_file:
    for word in sorted(idx):
        idx_words = ', '.join(idx[word])
        idx_line = "{}: {}".format(word, idx_words)
        print(idx_line, file=idx_file)

In [9]:
mappings = [('ph', 'f'), ('ght', 't'), ('^kn', 'n'), ('qu', 'kw'),
            ('[aeiou]+', 'a'), (r'(.)\1', r'\1')]

In [10]:
def signature(word):
    for patt, repl in mappings:
        word = re.sub(patt, repl, word)
    pieces = re.findall('[^aeiou]+', word)
    return ''.join(char for piece in pieces for char in sorted(piece))[:8]

In [11]:
signature('illefent')
# 'lfnt'

'lfnt'

In [12]:
signature('ebsekwieous')
# 'bskws'

'bskws'

In [13]:
signature('nuculerr')
# 'nclr'

'nclr'

In [14]:
signatures = nltk.Index((signature(w), w) for w in nltk.corpus.words.words())

In [15]:
signatures[signature('nuculerr')]
# ['anicular', 'inocular', 'nucellar', 'nuclear', 'unicolor', 'uniocular', 'unocular']

['anicular',
 'inocular',
 'nucellar',
 'nuclear',
 'unicolor',
 'uniocular',
 'unocular']

In [16]:
def rank(word, wordlist):
    ranked = sorted((nltk.edit_distance(word, w), w) for w in wordlist)
    return [word for (_, word) in ranked]

In [17]:
def fuzzy_spell(word):
    sig = signature(word)
    if sig in signatures:
        return rank(word, signatures[sig])
    else:
        return []

In [18]:
fuzzy_spell('illefent')
# ['olefiant', 'elephant', 'oliphant', 'elephanta']

['olefiant', 'elephant', 'oliphant', 'elephanta']

In [19]:
fuzzy_spell('ebsekwieous')
# ['obsequious']

['obsequious']

In [20]:
fuzzy_spell('nucular')
# ['anicular', 'inocular', 'nucellar', 'nuclear', 'unocular', 'uniocular', 'unicolor']

['anicular',
 'inocular',
 'nucellar',
 'nuclear',
 'unocular',
 'uniocular',
 'unicolor']